In [1]:
## a quick program to calculate expected and measured velocities 
## of Solar System objects from Rubin data

## 1 Sept 2024 -MEW

import numpy as np

In [9]:
## define our constants
G_au_yr = 39.422888 #G in au*(au/yr)^2/M_sun units
G_km_s = 887.1287008 #G in au*(km/s)^2/M_sun units
G_au_s = 3.96402313*10**(-14) #G in au*(au/s)^2/M_sun units
G_au_d = 2.95912741*10**(-4) #G in au*(au/day)^2/M_sun units

In [16]:
def keplerian_velocity(r, a):
    # this function allows us to calculate the predicted,
    # Keplerian velocity of an object using: 
    # v = sqrt(GM(2/r - 1/a))
    
    # note r = distance from Sun to object in AU
    # and a = semi-major axis in AU
    
    # note also that we are assuming M = M_sun + M_object ~ M_sun, 
    # and since M_sun is the unit of mass in the G definition
    # we are omitting it here. 

    # this returns the Keplerian velocity in AU/day
    return np.sqrt((G_au_d)*((2/r)-(1/a)))
    
    # this returns the Keplerian velocity in AU/yr, km/s, AU/s, and AU/day in that order
    # return np.sqrt((G_au_yr)*((2/r)-(1/a))), np.sqrt((G_km_s)*((2/r)-(1/a))), np.sqrt((G_au_s)*((2/r)-(1/a))), np.sqrt((G_au_d)*((2/r)-(1/a)))

In [17]:
def measured_velocity(vx, vy, vz):
    # this function calculates the total velocity from the x,y,z 
    # heliocentric velocities from the SSSource table
    
    # not totally sure what units these are in -- data product
    # document says that the heliocentric velocities are in "AU" 
    # whatever that means... I am thinking they are in AU/day, based on 
    # comparing Keplerian velocities calculated for various units, bc
    # AU/day agreed the best.
    
    return np.sqrt(vx**2 + vy**2 + vz**2)

In [18]:
# import a table of LSST DP03 simulation data
fname = 'table_dp03_catalogs_10yr.MPCORB-data.lsst.cloud.csv'
infile = open(fname, 'r')
    
rows = []
    
# this is a quick and rough way to load in the data -- remember to port it over 
# to a nicer, pandas version (similar to Brian's data loader) soon - this
# is just to get the ideas down
for line in infile.readlines()[1:]:
    l = line.split(',')
    rows.append(l)
        
#print(rows[0])

In [20]:
# now, calculate both the predicted velocities for Keplerian orbits
# as well as the measured heliocentric velocities in order to compare 
# them... 

# as you can see, the values in the final column (the measured velocities)
# are wildly different from the values in the first three columns (the Keplerian
# velocities in various units); not sure why this is, whether it is a unit
# conversion issue, a mistake in my math somewhere, or if my assumptions are 
# overall flawed... 

velocities = []
    
for cols in rows:
    # get heliocentric distance
    r = float(cols[7])

    # get the perihelion distance and eccentricity, and calculate
    # the semi-major axis
    q = float(cols[2])
    e = float(cols[1])
    a = q/(1-e)

    # compute the predicted Keplerian velocity in AU/day
    vk = keplerian_velocity(r, a)

    # retrieve the x,y,z heliocentric velocities
    vx = float(cols[8])
    vy = float(cols[9])
    vz = float(cols[10])

    # calculate the measured heliocentric velocity
    v_actual = measured_velocity(vx, vy, vz)

    v = [vk, v_actual, v_actual-vk]
    velocities.append(v)
    print(v)

[0.009574696080806022, 0.009580795267158717, 6.099186352694422e-06]
[0.009735288628211257, 0.00974037488827911, 5.086260067852302e-06]
[0.009684695438230598, 0.009695557318576126, 1.0861880345528277e-05]
[0.012227672591931506, 0.012234404853190804, 6.732261259297795e-06]
[0.010188493878884621, 0.010197885067552823, 9.391188668201808e-06]
[0.01227183468858238, 0.012278447839062557, 6.613150480177729e-06]
[0.00975730954792141, 0.009765063770567657, 7.754222646247366e-06]
[0.012337948425207988, 0.012342520428709765, 4.572003501776556e-06]
[0.01230447275619438, 0.012310964889333422, 6.492133139041761e-06]
[0.009988311030563322, 0.009993578493078014, 5.2674625146912485e-06]
[0.009496888663809929, 0.00950085763849802, 3.968974688090682e-06]
[0.0100236214775208, 0.010028824463055176, 5.20298553437562e-06]
[0.009779989887431843, 0.009790716365748806, 1.0726478316963259e-05]
[0.009873209338571094, 0.009878372195476354, 5.162856905259819e-06]
[0.010122686697861295, 0.010128005034926337, 5.318337

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# sample calculation for one object, written before I loaded in the CSV file
# above -- feel free to disregard as it is outdated and basically duplicates the above

# get r from heliocentricDist in SSSource table
r = 2.8450608
    
# get a by calculating it from q and e from MPCORB table
q = 2.169822915818627 #in AU
e = 0.14689253115098788

a = q/(1-e)
    
vk_au, vk_km, vk_au_s = keplerian_velocity(r, a)
    
# get heliocentric vx, vy, and vz from SSSource table
vx = 0.0065061725
vy = -0.0066827396
vz = -0.0021914262
    
v_actual = measured_velocity(vx, vy, vz)
    
print("keplerian velocity (au/yr): "+str(vk_au))
print("keplerian velocity (km/s): "+str(vk_km))
print("keplerian velocity (au/s): "+str(vk_au_s))
print("measured velocity (au/?): "+str(v_actual))